In [1]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

In [2]:
books_url = "https://books.toscrape.com" 
books_url
# + "/catalogue/soumission_998/"

'https://books.toscrape.com'

In [3]:
url_client = urlopen(books_url)
url_client

In [4]:
books_page = url_client.read()
books_html = bs(books_page, "html.parser")

In [5]:
main_div = books_html.findAll("li", {"class": "col-xs-6 col-sm-4 col-md-3 col-lg-3"})

In [6]:
len(main_div)

20

In [7]:
main_div[0].a['href']


'catalogue/a-light-in-the-attic_1000/index.html'

In [8]:
"https://books.toscrape.com/" + main_div[0].a['href'] 

'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

In [9]:
product_link = "https://books.toscrape.com/" + main_div[1].a['href'] 
product_req = requests.get(product_link)

In [10]:
product_html = bs(product_req.text, "html.parser")

In [11]:
product_detail = product_html.find("div", {"class": "col-sm-6 product_main"})
product_detail

<div class="col-sm-6 product_main">
<h1>Tipping the Velvet</h1>
<p class="price_color">Â£53.74</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock (20 available)
    
</p>
<p class="star-rating One">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<!-- <small><a href="/catalogue/tipping-the-velvet_999/reviews/">
        
                
                    0 customer reviews
                
        </a></small>
         --> 


<!-- 
    <a id="write_review" href="/catalogue/tipping-the-velvet_999/reviews/add/#addreview" class="btn btn-success btn-sm">
        Write a review
    </a>

 --></p>
<hr/>
<div class="alert alert-warning" role="alert"><strong>Warning!</strong> This is a demo website for web scraping purposes. Prices and ratings here were randomly assigned and have no real meaning.</div>
</div>

In [12]:
title = product_detail.h1.text
title

'Tipping the Velvet'

In [13]:
stars = product_detail.find("p", {"class": "star-rating"})
star_class = stars.get("class")
#  ['star-rating', 'Three'])
star_text = star_class[1] if len(star_class) > 1 else None

star_mapping = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}
star_number = star_mapping.get(star_text, 0)
star_number

1

In [14]:
availability = product_detail.find("p" , {"class": "instock availability"}).text
availability = availability.replace('\n', '').strip()
availability


'In stock (20 available)'

In [15]:
price = product_detail.find("p", {"class": "price_color"}).text
price = price.replace("Â", "")
price

'£53.74'

In [16]:
article = product_html.find("article", {"class": "product_page"})
# <p> tags with no class attribute
p_no_class = [p for p in article.find_all("p") if not p.has_attr("class")]
p_no_class[0].text

'"Erotic and absorbing...Written with starling power."--"The New York Times Book Review " Nan King, an oyster girl, is captivated by the music hall phenomenon Kitty Butler, a male impersonator extraordinaire treading the boards in Canterbury. Through a friend at the box office, Nan manages to visit all her shows and finally meet her heroine. Soon after, she becomes Kitty\'s "Erotic and absorbing...Written with starling power."--"The New York Times Book Review " Nan King, an oyster girl, is captivated by the music hall phenomenon Kitty Butler, a male impersonator extraordinaire treading the boards in Canterbury. Through a friend at the box office, Nan manages to visit all her shows and finally meet her heroine. Soon after, she becomes Kitty\'s dresser and the two head for the bright lights of Leicester Square where they begin a glittering career as music-hall stars in an all-singing and dancing double act. At the same time, behind closed doors, they admit their attraction to each other 

In [17]:
import csv
data_list = []

for book in main_div:
    
    book_link = books_url + "/" + book.a['href']
    product_req = requests.get(book_link)
    product_html = bs(product_req.text, "html.parser")
    product_detail = product_html.find("div", {"class": "col-sm-6 product_main"})
    
    
    title = product_detail.h1.text.strip()
    price = product_detail.find("p", {"class": "price_color"}).text.replace("Â", "").strip()
    stars = product_detail.find("p", {"class": "star-rating"})
    star_class = stars.get("class")
    star_text = star_class[1] if len(star_class) > 1 else None
    star_mapping = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
    star_number = star_mapping.get(star_text, 0)
    availability = product_detail.find("p", {"class": "instock availability"}).text.replace('\n', '').strip()
    article = product_html.find("article", {"class": "product_page"})
    p_no_class = [p for p in article.find_all("p") if not p.has_attr("class")]
    description = p_no_class[0].text.strip() if p_no_class else ""
    
    data_list.append({
        "title": title,
        "price": price,
        "stars": star_number,
        "availability": availability,
        "description": description,
        "product_link": book_link
    })


fieldnames = ["title", "price", "stars", "availability", "description", "product_link"]

with open("book_details.csv", mode="w", encoding="utf-8", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for book in data_list:
        writer.writerow(book)

In [18]:
import pandas as pd
df = pd.read_csv("book_details.csv")
df.head(10)

,title,price,stars,availability,description,product_link
0,A Light in the Attic,£51.77,3,In stock (22 available),It's hard to imagine a world without A Light i...,https://books.toscrape.com/catalogue/a-light-i...
1,Tipping the Velvet,£53.74,1,In stock (20 available),"""Erotic and absorbing...Written with starling ...",https://books.toscrape.com/catalogue/tipping-t...
2,Soumission,£50.10,1,In stock (20 available),"Dans une France assez proche de la nÃ´tre, un ...",https://books.toscrape.com/catalogue/soumissio...
3,Sharp Objects,£47.82,4,In stock (20 available),"WICKED above her hipbone, GIRL across her hear...",https://books.toscrape.com/catalogue/sharp-obj...
4,Sapiens: A Brief History of Humankind,£54.23,5,In stock (20 available),From a renowned historian comes a groundbreaki...,https://books.toscrape.com/catalogue/sapiens-a...
5,The Requiem Red,£22.65,1,In stock (19 available),Patient Twenty-nine.A monster roams the halls ...,https://books.toscrape.com/catalogue/the-requi...
6,The Dirty Little Secrets of Getting Your Dream...,£33.34,4,In stock (19 available),Drawing on his extensive experience evaluating...,https://books.toscrape.com/catalogue/the-dirty...
7,The Coming Woman: A Novel Based on the Life of...,£17.93,3,In stock (19 available),"""If you have a heart, if you have a soul, Kare...",https://books.toscrape.com/catalogue/the-comin...
8,The Boys in the Boat: Nine Americans and Their...,£22.60,4,In stock (19 available),For readers of Laura Hillenbrand's Seabiscuit ...,https://books.toscrape.com/catalogue/the-boys-...
9,The Black Maria,£52.15,1,In stock (19 available),"Praise for Aracelis Girmay:""[Girmay's] every l...",https://books.toscrape.com/catalogue/the-black...


In [ ]:
import json
with open("book_details.json", "w", encoding="utf-8") as f:
    json.dump(data_list, f, ensure_ascii=False, indent=2)